In [1]:
import os
import torch
import torch.nn as nn
import time
from torch.autograd import grad


from mymodule.model import resnet34
from mymodule.pif.influence_functions_new import get_gradient, tracin_get
from mymodule.data_get import dataset_get, dataset_category_get


In [2]:
category_num = 0
img_all_train, img_all_test = dataset_category_get(category_num=category_num)

In [3]:
img_all_train.shape

torch.Size([500, 3, 224, 224])

In [4]:
img_all_test.shape

torch.Size([100, 3, 224, 224])

In [5]:
for i in range(10):
    print(img_all_train[0][0][0][i])

tensor(0.7248)
tensor(0.7419)
tensor(0.7591)
tensor(0.7762)
tensor(0.7933)
tensor(0.8276)
tensor(0.8618)
tensor(0.8961)
tensor(0.9303)
tensor(0.9646)


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
net = resnet34()
in_channel = net.fc.in_features
net.fc = nn.Linear(in_channel, 10)
net.to(device) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(64, 

In [8]:
# load pretrain weights
# download url: https://download.pytorch.org/models/resnet34-333f7ec4.pth

In [1]:
model_weight_path = "pth_file/resNet34_epoch1.pth"
model_weight_path = "pth_file2/resNet34_tracin_120.pth"
assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
net.load_state_dict(torch.load(model_weight_path, map_location=device))
net.to(device)

In [1]:
from torch import nn

In [2]:
loss_fn = nn.CrossEntropyLoss()
img_all_train = img_all_train.view(500, 1, 3, 224, 224)
img_all_test = img_all_test.view(100, 1, 3, 224, 224)
label_train = torch.zeros(1).long()
logits_train = net(img_all_train[0].to(device))
loss_train = loss_fn(logits_train, label_train.to(device))
grad_z_train = grad(loss_train, net.parameters())
grad_z_train = get_gradient(grad_z_train, net)
score_list = []
time_start = time.perf_counter()
for i in range(10000):
    label_test = torch.zeros(1).long()
    label_test[0] = 0
    logits_test = net(img_all_test[i].to(device))
    loss_test = loss_fn(logits_test, label_test.to(device))
    grad_z_test = grad(loss_test, net.parameters())
    grad_z_test = get_gradient(grad_z_test, net)
    score = tracin_get(grad_z_test, grad_z_train)
    score_list.append(float(score))
    print(score)
print('%f s' % (time.perf_counter() - time_start))
print(score_list)

NameError: name 'img_all_train' is not defined